In [11]:
import praw
import json
from datetime import datetime, timezone
from dotenv import load_dotenv, dotenv_values 

from tqdm import tqdm

import pandas as pd
import os
from scraper import RedditScraper
load_dotenv()

True

In [12]:
client_id = os.getenv("CLIENT_ID")
client_secret=os.getenv("CLIENT_SECRET")
user_agent=os.getenv("USER_AGENT")
scraper = RedditScraper(client_id, client_secret, user_agent)

## User comment fetch

In [13]:
subreddits = [
    "books"
] 

In [14]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "Data"))
USERLIST_FILEPATH = os.path.join(BASE_DIR, 'Reddit Scrape/unique_users.csv')
USERCOMM_SAVEPATH = os.path.join(BASE_DIR, 'usercomments')

os.makedirs(USERCOMM_SAVEPATH, exist_ok=True)

In [15]:
users_df = pd.read_csv(USERLIST_FILEPATH)
users_list = users_df['user'].tolist()
# users_list = ["1barefootmaniac"]

In [23]:
def pull_user_comments(userslist, subreddits, save_basepath, limit):

    for user in tqdm(userslist, desc="Scraping user comments"):
        save_file = f'{user}_usercomments.json'
        save_path = os.path.join(save_basepath, save_file)

        posts = scraper.scrape_user_political_comments(user, subreddits, limit=limit)

        with open(save_path, "w") as f:
            json.dump(posts, f, indent=4)

        # Print progress for debugging/logging (optional)
        print(f"Comments saved to {save_path}")

In [25]:
LIMIT = None
pull_user_comments(users_list, subreddits, USERCOMM_SAVEPATH, limit=LIMIT)

Scraping user comments:   0%|          | 0/29911 [00:00<?, ?it/s]

{'television', 'coaxedintoasnafu', 'dontflinch', 'watchpeoplesurvive', 'TwoHotTakes', 'SipsTea', 'stuffyoushouldknow', 'DavidYurman', 'writing', 'mildlyinteresting', 'itcouldhappenhere', 'Showerthoughts', 'self', 'borderlands3', 'doordash', 'Millennials', 'Warhammer40k', 'tampa', 'me_irl', 'holyshit', 'ExpectationVsReality', 'aiArt', 'horror', 'MageErrant', 'NameNerdCirclejerk', 'DnD5e', 'meme', 'csgo', 'ShermanPosting', 'FluentInFinance', 'pokemon', 'kickopenthedoor', 'geography', 'AskReddit', 'Grimdank', 'skyrim', 'chyberpunk', 'AITA_WIBTA_PUBLIC', '13or30', 'trueratediscussions', 'HongKong', 'PoliticalDiscussion', 'popping', 'JustUnsubbed', 'Delta_Emulator', 'blankies', 'MobileAL', 'BridgertonNetflix', 'ArtefactPorn', 'SocialistGaming', 'Firefighting', 'takis', 'moviecritic', 'DarkTide', 'penguinz0', 'comics', 'whowouldwin', 'ShaneGillis', 'menswear', 'unpopularopinion', 'science', 'Overwatch', 'youtube', 'ExplainTheJoke', 'OddSatisfying', 'ForwardsFromKlandma', 'offbeat', 'Fantasy'

Scraping user comments:   0%|          | 1/29911 [00:28<239:38:45, 28.84s/it]

Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/---Sanguine---_usercomments.json
{'ask', 'dominoes', 'AskOldPeople', 'mildlyinteresting', 'videos', 'rant', 'MusicRecommendations', 'houstonwade', 'technology', 'sports', 'law', 'self', 'Millennials', 'bronco', 'PastAndPresentPics', 'boston', 'bostontrees', 'geography', 'AskReddit', 'Cooking', '13or30', 'gifs', 'MurderedByWords', 'the_everything_bubble', 'WidespreadPanic', 'iwatchedanoldmovie', '90sHipHop', 'reggae', 'inflation', 'Wellworn', 'Bumperstickers', 'deadandcompany', 'BeastieBoys', 'DIY', 'Pennsylvania', 'Tuckahoe10707', 'hotsauce', 'UpliftingNews', 'psychedelicrock', 'MastersoftheAir', 'Disneyland', 'technews', 'Concerts', 'funk', 'ClassicRock', 'GenX', 'Damnthatsinteresting', 'creepy', 'Futurology', 'AskHistory', 'BudgetAudiophile', 'newengland', 'Flooring', 'asheville', 'tiedye', 'Rucking', 'LSD', 'fastfood', 'BBQ', 'ledzeppelin', 'gratefuldead', 'texas', 'blackstonegriddle', 'ne

Scraping user comments:   0%|          | 2/29911 [00:57<237:28:27, 28.58s/it]

...

Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/84lele_usercomments.json
{'northcounty', 'sandiego', 'books', 'SanDiegan'}


Scraping user comments:   1%|          | 322/29911 [1:14:10<162:35:14, 19.78s/it]

Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/858Librarian_usercomments.json


Scraping user comments:   1%|          | 322/29911 [1:14:14<113:42:41, 13.83s/it]


TooManyRequests: received 429 HTTP response

## Post fetch

In [ ]:
# post_url = "https://www.reddit.com/r/politics/comments/1gc7k0y/bcso_voter_punches_poll_worker_when_reminded/"

# # Retrieve comments
# comments = scraper.get_comments_from_post(post_url)

# # Save comments to a JSON file
# with open("../Data/comments_graph.json", "w") as f:
#     json.dump(comments, f, indent=4)

# print("Comments saved to comments.json")